In [1]:
import os
os.chdir('../..')
!pwd

/Users/aykut/Desktop/aykut/laughter-synthesis


In [8]:
%load_ext autoreload
%autoreload 2

import numpy as np
import sounddevice as sd
import utils.audio as audio
from IPython.display import Audio

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
SR = 44100
DURATION_IN_SEC = 5

In [81]:
x = audio.spawnClaps(10, SR, DURATION_IN_SEC)

In [44]:
Audio(x.astype(np.float32), rate=SR)

In [49]:
s = 0

def callback(outdata, frames, time, status):
    global s
    if status:
        print(status)
    outdata[:] = x.T[s:s + frames]
    s += frames
    return 


In [50]:
sd_os = sd.OutputStream(samplerate=SR, channels=2, callback=callback, dtype='float32')

In [51]:
sd_os.start()

From cffi callback <function _StreamBase.__init__.<locals>.callback_ptr at 0x7ff92d0473a0>:
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sounddevice.py", line 856, in callback_ptr
    return _wrap_callback(callback, data, frames, time, status)
  File "/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sounddevice.py", line 2678, in _wrap_callback
    callback(*args)
  File "<ipython-input-49-611d6f264577>", line 7, in callback
ValueError: could not broadcast input array from shape (22,2) into shape (34,2)


In [48]:
sd_os.stop()

From cffi callback <function _StreamBase.__init__.<locals>.callback_ptr at 0x7ff92d0334c0>:
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sounddevice.py", line 856, in callback_ptr
    return _wrap_callback(callback, data, frames, time, status)
  File "/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sounddevice.py", line 2678, in _wrap_callback
    callback(*args)
  File "<ipython-input-45-e0ba80915730>", line 7, in callback
ValueError: could not broadcast input array from shape (22,2) into shape (34,2)


In [73]:
import queue
import threading
SOUNDWAVE_BIN_SIZE = 20

In [107]:
def start_stream(audio, stream_frame):
    def callback(outdata, frames, time, status):
        if status:
            print(status)
            
        try:
            data = q.get_nowait()
        except queue.Empty:
            print('Buffer is empty: increase buffersize?')
            raise sd.CallbackAbort
        if len(data) < len(outdata):
            outdata[:len(data)] = data
            outdata[len(data):] = b'\x00' * (len(outdata) - len(data))
            raise sd.CallbackStop
        else:
            outdata[:] = data
            
    q = queue.Queue(maxsize=SOUNDWAVE_BIN_SIZE)
    event = threading.Event()
    
    stream = sd.OutputStream(
        samplerate=SR, 
        channels=2, 
        callback=callback, 
        blocksize=SOUNDWAVE_BIN_SIZE,
        dtype='float32'
    )
    
    with stream:
        data = audio.T[stream_frame:stream_frame + SOUNDWAVE_BIN_SIZE]
        while len(data) != 0:
            stream_frame += SOUNDWAVE_BIN_SIZE
            q.put(data)
            data = audio.T[stream_frame:stream_frame + SOUNDWAVE_BIN_SIZE]


In [108]:
start_stream(x, 0)

<generator object start_stream at 0x7ff92dead430>

In [117]:
109148 / 22050

4.950022675736961

In [118]:
np.arange(99, 100)

array([99])

In [119]:
w = np.zeros(100)

In [120]:
w[99:]

array([0.])

In [236]:
import utils.audio as utils_audio

In [238]:
!pwd

/Users/aykut/Desktop/aykut/laughter-synthesis


In [277]:
n_person = 10
whistling_intensity = 100

In [278]:
claps = utils_audio.spawnClaps(n_person, SR, DURATION_IN_SEC)

In [279]:
root_whistle = utils_audio.spawnWhistles(
    SR, 
    can_radius=200, 
    pea_radius=50, 
    bump_radius=150, 
    fipple_freq_mod=.35, 
    fipple_gain_mod=.35, 
    noise_gain=.2, 
    base_freq=5000, 
    sine_rate=4000, 
    pole=.95, 
    norm_can_loss=.97,
    load=True, 
    load_path='/Users/aykut/Desktop/aykut/laughter-synthesis/Hand-Whistle.wav'
)

In [296]:
whistles = np.zeros_like(claps)
n_person_whistling = int(n_person * whistling_intensity / 100)
for _ in range(n_person_whistling):
    start = int(np.random.uniform(0, DURATION_IN_SEC - 1) * SR)
    direction = np.random.randint(0, 2)
    whistle = utils_audio.changePitch(root_whistle, SR, np.random.uniform(-3, 3))

    whistles[direction, start:start + len(whistle)] += whistle

In [297]:
audio = claps + whistles * 200

In [298]:
Audio(audio, rate=SR)

In [274]:
(DURATION_IN_SEC - 1) * SR

176400

In [275]:
start

69504

In [268]:
start + len(whistle)

124579

In [276]:
whistles[direction, start:start + len(whistle)].shape

(40746,)

In [264]:
whistle.squeeze().shape

(44100,)

In [256]:
direction

1

In [259]:
start + len(whistle)

95701